In [1]:
import numpy as np
import pandas as pd
import glob
import psycopg2
from sql_queries import *
from dwh import *

# Intro

Analyse a single .csv to get the column names and build the queries

In [2]:
df = pd.read_csv('./data/1/1.csv')  

/home/marta/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (77,84,85) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df['ID_KEY'] = str(df['MONTH'].values[0])+'_'+df.index.map(str)

In [4]:
df = df.rename(columns={'YEAR':'FL_YEAR','QUARTER':'FL_QUARTER','MONTH':'FL_MONTH','Unnamed: 109':'UNNAMED'})

## INSERT query
Build an insert query for the staging table, which contains all the info in the .csv file

In [5]:
columns = list(df.columns)
query="\n INSERT INTO staging ( "
end=") VALUES ("
for el in columns:
    query+=" \n "+el+","
    end+="%s,"
insert_query = query[:-1]+end[:-1]+") ON CONFLICT ON CONSTRAINT staging_pkey DO NOTHING\n"

# Create DB, staging table and fill it

In [6]:
def create_database(dropdb=False):
    '''
    Function to create the DB
     Key arg: dropdb. Default is False
    
     - connect to the db
     - drop DB if exists 
     - create DB
 
    Returns a cursor and a connection to the DB
    '''
    conn = psycopg2.connect("host={} dbname={} user={} password={}".format(myhost,mydb,myuser,mypass))
    conn.autocommit = True
    cur = conn.cursor()
    if dropdb:
        cur.execute("DROP DATABASE IF EXISTS planesdb")
        cur.execute("CREATE DATABASE planesdb WITH ENCODING 'utf8' TEMPLATE template0")
    
    conn.close()
    conn = psycopg2.connect("host={} dbname=planesdb user={} password={}".format(myhost,myuser,mypass))
    cur = conn.cursor()
    
    return cur, conn

In [7]:
cur, conn = create_database()

Drops the table if it exists and creates it again

In [8]:
def create_table():
    '''
    Function to create the table
     - drop table if exists
     - create table
    '''
    cur.execute(staging_table_drop)
    cur.execute(staging_table)
    conn.commit()

In [9]:
# CAREFUL!! creating the staging table erases it first, if it already exists...and refilling it will take looong
# create_table()

Fills in the table from the .csv file.

In [10]:
def insert_csv(path, test=False):
    '''
    Function to fill in the staging table
    Key args: 
    - path: the path to the csv file
    - test: if True, does not iterate over the whole csv but only over the first 5 rows
    '''
    df = pd.read_csv(path)  
    df['ID_KEY'] = str(df['MONTH'].values[0])+'_'+df.index.map(str)
    df = df.rename(columns={'YEAR':'FL_YEAR','QUARTER':'FL_QUARTER','MONTH':'FL_MONTH','Unnamed: 109':'UNNAMED'})
    if test:
        df = df.head()
    for index, row in df.iterrows():
        #could probably do it better
        values = [value if (type(value)==str or type(value)==float) else value.item() for value in df.iloc[index,:].values]
        cur.execute(insert_query,values)
        conn.commit()

In [11]:
refill = False

if refill==True:
    path = r'./data/all' # use your path
    all_files = glob.glob(path + "/*.csv")

    li = []
    
    for filename in all_files:
        print(filename)
        insert_csv(filename)

# Rearrange the DB
Rearranging the DB to the star schema, as explained in the README.

In [12]:
def drop_star_tables():
    '''
    Function to create the tables of the star schema DB. For each table:
     - drop table if exists
     - create table
    '''
    for query in drop_table_queries:
        cur.execute(query)
        conn.commit()

In [13]:
# drop_star_tables()

In [14]:
def create_star_tables():
    '''
    Function to create the tables of the star schema DB. For each table:
     - drop table if exists
     - create table
    '''
    for query in create_table_queries:
        cur.execute(query)
        conn.commit()

In [15]:
# create_star_tables()

In [16]:
def fill_star_tables():
    '''
    Function to create the tables of the star schema DB. For each table:
     - drop table if exists
     - create table
    '''
    for query in insert_queries:
        cur.execute(query)
        conn.commit()

In [17]:
# fill_star_tables()

In [18]:
conn.commit()

In [19]:
# conn.close()